# Dependencies

In [48]:
from ai_framework.GenAILLM import (
    EnumGenAIModelsIdsOpenAI,
    EnumGenAIPlatforms,
    GenAILLM
)
from ai_framework.LoggingAndTelemetry import EnumLogs, Logging
from ai_framework.Embedder import Embedder
from ai_framework.GenAINodeChain import *
import os
from dotenv import load_dotenv

# Variables

In [49]:
load_dotenv()

True

In [50]:
video_id = 'https://www.youtube.com/watch?v=Q6vk6rU6DfU'
languages = ['ja']

In [51]:
verbose_level: EnumLogs = EnumLogs.LOG_LEVEL_DEBUG
verify_ssl = False

gen_ai_llm_4k = GenAILLM(
    platform=EnumGenAIPlatforms.PLATFORM_OPENAI,
    model_id=EnumGenAIModelsIdsOpenAI.MODEL_CHAT_GPT_4,
    parameters_inference={
        "max_tokens": 2048,
        "temperature": 0,
        "top_p": 0,
        # "stop_sequences": ["User"],
        # "max_tokens_to_sample": 1024,
        # "top_k": 5,
    },
    platform_configuration={
        "OPENAI_API_KEY": os.environ["OPENAI_API_KEY"],
    },
    verify_ssl=verify_ssl,
    verbose_level=verbose_level,
)

gen_ai_llm_16k = GenAILLM(
    platform=EnumGenAIPlatforms.PLATFORM_OPENAI,
    model_id=EnumGenAIModelsIdsOpenAI.MODEL_CHAT_GPT_3_5_TURBO_16k,
    parameters_inference={
        "max_tokens": 4096,
        "temperature": 0,
        "top_p": 0,
        # "stop_sequences": ["User"],
        # "max_tokens_to_sample": 1024,
        # "top_k": 5,
    },
    platform_configuration={
        "OPENAI_API_KEY": os.environ["OPENAI_API_KEY"],
    },
    verify_ssl=verify_ssl,
    verbose_level=verbose_level,
)

gen_ai_llm_128k = GenAILLM(
    platform=EnumGenAIPlatforms.PLATFORM_OPENAI,
    model_id=EnumGenAIModelsIdsOpenAI.MODEL_CHAT_GPT_4_TURBO_128k,
    parameters_inference={
        "max_tokens": 4096,
        "temperature": 0,
        "top_p": 0,
        # "stop_sequences": ["User"],
        # "max_tokens_to_sample": 1024,
        # "top_k": 5,
    },
    platform_configuration={
        "OPENAI_API_KEY": os.environ["OPENAI_API_KEY"],
    },
    verify_ssl=verify_ssl,
    verbose_level=verbose_level,
)

2024-03-13 22:00:54.852127-05:00: Modelo EnumGenAIModelsIdsOpenAI.MODEL_CHAT_GPT_4 construido
2024-03-13 22:00:54.853128-05:00: Modelo EnumGenAIModelsIdsOpenAI.MODEL_CHAT_GPT_3_5_TURBO_16k construido
2024-03-13 22:00:54.855126-05:00: Modelo EnumGenAIModelsIdsOpenAI.MODEL_CHAT_GPT_4_TURBO_128k construido


In [52]:
gen_ai_memory = GenAIMemory()

# Code

In [53]:
from youtube_transcript_api import YouTubeTranscriptApi

def extract_youtube_id(url_video):
    return url_video.split("watch?v=")[-1]

def get_transcript(video_id, languages=['de', 'en']):

    try:
        output = YouTubeTranscriptApi.get_transcript(video_id, languages=languages)
        texts = [data['text'] for data in output]
        return ' '.join(texts)
    except Exception as e:
        print(e)
        return None

# Aquí debes reemplazar 'your_video_id' con el ID de tu video de YouTube

video_id = extract_youtube_id(video_id)
transcript = get_transcript(video_id, languages=languages)
file_name_transcript = f"./data/input/transcript_{video_id}.txt"

with open(file_name_transcript, "w") as f_out:
    f_out.write(transcript)

In [54]:
def func_forward_and_save_chain(
    node: GenAINodeChain, shared_state, gen_ai_memory
):
    Logging.log(f"RUNNING: {node.id}", node.verbose_level)
    chain_result = node.get_chain_result(shared_state, gen_ai_memory)
    return {
        **shared_state,
        "intermediate_steps_bp": [(node.id, "default")],
        **chain_result,
        "node_outcome": chain_result,
    }

In [55]:
node_topic_understanding = GenAINodeChain("topic_understanding",
    gen_ai_llm_16k,
    gen_ai_prompt=GenAIPrompt(
        """{system_prompt}
    '''
    {transcript}
    '''
    {format_instructions}
    """,
        partials={
            "system_prompt": """Recibirás una transcripción de un video de youtube en un idioma diferente al español. Deberás extraer el tema y adecuar todo ese texto
            desordenado que puede ocurrir entre varias personas de forma más estructurada por título, subtítulos, secciones, etc. Adicionalmente expande el contenido, no te limites a lo que dice la transcipción
            Por ejemplo si la transcripción es:
            '''Presentador: ¡Hola a todos! Bienvenidos de nuevo a nuestro canal. En el día de hoy vamos a hablar sobre las últimas tendencias en moda primavera-verano. Como pueden ver, los colores vibrantes y los estampados florales están muy de moda esta temporada. También estamos viendo una tendencia hacia los cortes asimétricos y los tejidos ligeros. En cuanto a los accesorios, los sombreros de ala ancha y las gafas de sol retro son imprescindibles para complementar cualquier look primaveral. ¡Esperamos que estos consejos les sean útiles para armar sus outfits esta temporada! No olviden suscribirse para más contenido de moda. ¡Hasta la próxima!
            '''
            Tu salida puede ser:
            {
                "tema": "Tendencias de moda primavera-verano",
                "guion": "Introducción\nPresentación del video y bienvenida al canal.\nColores y estampados de moda\nDescripción de los colores vibrantes y estampados florales como tendencia.\nCortes y tejidos\n\nDiscusión sobre la popularidad de los cortes asimétricos y tejidos ligeros.\nAccesorios imprescindibles\n\nMención de los sombreros de ala ancha y las gafas de sol retro como accesorios clave.\nConsejos finales y llamado a la acción\n\nRecomendaciones finales para armar outfits de moda y recordatorio para suscribirse al canal."
            }

            Recuerda hacerlo todo en español
        """
        },
    ),
    gen_ai_output_parser=GenAIOutputParser([
        {
            "name": "tema",
            "description": "aquí extraeras el tema de lo hablado en el video. Debe ser un texto corto"
        },
        {
            "name": "guion",
            "description": "aquí formatearás el texto de la transcripción de una forma más ordenada y dividida por títulos y subtítulos. Expandiendo el tema usando tus conocimientos"
        },
    ], natural_language_key="guion"),
    func_invoke=func_forward_and_save_chain,
    verbose_level=verbose_level,
)

node_topic_understanding_out = node_topic_understanding.invoke({
    "transcript": transcript
}, gen_ai_memory)
print(node_topic_understanding_out.keys())

2024-03-13 22:00:56.459581-05:00: RUNNING: topic_understanding


> Entering new LLMChain chain...
Prompt after formatting:
Recibirás una transcripción de un video de youtube en un idioma diferente al español. Deberás extraer el tema y adecuar todo ese texto
            desordenado que puede ocurrir entre varias personas de forma más estructurada por título, subtítulos, secciones, etc. Adicionalmente expande el contenido, no te limites a lo que dice la transcipción
            Por ejemplo si la transcripción es:
            '''Presentador: ¡Hola a todos! Bienvenidos de nuevo a nuestro canal. En el día de hoy vamos a hablar sobre las últimas tendencias en moda primavera-verano. Como pueden ver, los colores vibrantes y los estampados florales están muy de moda esta temporada. También estamos viendo una tendencia hacia los cortes asimétricos y los tejidos ligeros. En cuanto a los accesorios, los sombreros de ala ancha y las gafas de sol retro son imprescindibles para complementar cualquier

In [56]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(node_topic_understanding_out["output_topic_understanding"], 'lxml')

final_answer = soup.find('final-answer')
tema = final_answer.find('tema').text.replace(":", "").replace('"', '')
guion = final_answer.find('guion').text
print(tema, guion)

Superación de obstáculos y miedo en la película "Curva" 

        La película "Curva" muestra a una mujer que se encuentra en una situación extrema, enfrentando sus miedos y desafíos.
        
        
        La protagonista se despierta en un lugar desconocido, enfrentando un abismo sin fin en un terreno inclinado y peligroso. Encuentra un misterioso cuaderno que aumenta su miedo y ansiedad.
        
        
        A pesar de la dificultad de la situación, la mujer intenta escalar la pared inclinada, enfrentando el miedo y la frustración. Cada intento la lastima más, pero sigue adelante.
        
        
        La película refleja las emociones instintivas de miedo y desesperación que experimenta la mujer en la pared curva. Aunque aparentemente trata sobre la lucha por la supervivencia, en realidad transmite un mensaje más profundo sobre el dolor psicológico y la superación personal.
        
        
        La escena de la lluvia intensifica el desafío y simboliza la negativida

In [57]:
from langchain_community.utilities import GoogleSerperAPIWrapper
search = GoogleSerperAPIWrapper()
search_result = search.results(tema)

organic_results = search_result['organic']

results_string = ""
for organic_result in organic_results:
    results_string += f"Titulo:{organic_result['title']}. Contenido:{organic_result['snippet']}\n"
print(results_string)

Titulo:Y si viene una curva , acelera #cima #proceso #obstaculos ... - TikTok. Contenido:Video de TikTok de MARENGO (@pieromarengocordova): «Y si viene una curva , acelera #cima ...
Titulo:Estrategias Para Superar Estancamientos Y Obstáculos .... Contenido:... curva y superar los desafíos.Al adoptar la innovación, las empresas pueden desarrollar nuevos productos, servicios y procesos que pueden ayudarlos a superar ...
Titulo:Las 10 mejores películas de deporte. Contenido:Nos adentramos en la intensidad de las carreras, donde cada curva, adelantamiento y estrategia puede marcar la diferencia entre el éxito y el ...
Titulo:Miedo Desconocido - FasterCapital. Contenido:... curva de adopción de tecnología.Si bien pueden ... Uno de los mayores obstáculos para probar cosas nuevas es el miedo. ... superar las barreras que nos frenan.
Titulo:Cómo superar el miedo a morir a través de la terapia psicológica. Contenido:La curva del estrés: ¿en qué consiste ...
Titulo:Nana Santaella on LinkedIn: #c

In [60]:
node_video_content_generation = GenAINodeChain("video_content_generation",
    gen_ai_llm_128k,
    gen_ai_prompt=GenAIPrompt(
        """{system_prompt}
    Entendimiento del transcript original:
    tema: {tema}
    resumen del transcript original: {guion}

    Transcript:
    {transcript}

    Información adicional del transcript original: 
    {results_string}
    """,
        partials={
            "system_prompt": """A partir del siguiente entendimiento de un transcript de un video adicional y de un contexto adicional obtenido
            de una búsqueda en Google. Realiza un texto en español para un video entretenido de Youtube. Con una duración alrededor de 8minutos
            NO agregues a un presentador en tu texto como un guión, agrega el texto tal cual será leido por un humano.
            Si tienes ideas de gags, chistes o similar ponlos en una etiqueta especial.
            Habla de todos los temas en la información adicional, genera un texto largo
            """,
        },
    ),
    gen_ai_output_parser=None,
    func_invoke=func_forward_and_save_chain,
    verbose_level=verbose_level,
)

node_video_content_generation_out = node_video_content_generation.invoke({
    "tema": tema,
    "guion": guion,
    "results_string": results_string,
    "transcript": transcript
}, gen_ai_memory)

file_name_script = f"./data/output/video_{tema}.txt"

final_output = node_video_content_generation_out[node_video_content_generation.output_key]

with open(file_name_script, "w") as f_out:
    f_out.write(final_output)

2024-03-13 22:03:26.610957-05:00: RUNNING: video_content_generation


> Entering new LLMChain chain...
Prompt after formatting:
A partir del siguiente entendimiento de un transcript de un video adicional y de un contexto adicional obtenido
            de una búsqueda en Google. Realiza un texto en español para un video entretenido de Youtube. Con una duración alrededor de 8minutos
            NO agregues a un presentador en tu texto como un guión, agrega el texto tal cual será leido por un humano.
            Si tienes ideas de gags, chistes o similar ponlos en una etiqueta especial.
            Habla de todos los temas en la información adicional, genera un texto largo
            
    Entendimiento del transcript original:
    tema: Superación de obstáculos y miedo en la película Curva
    resumen del transcript original: 

        La película "Curva" muestra a una mujer que se encuentra en una situación extrema, enfrentando sus miedos y desafíos.
        
        
        La protago

In [61]:
from gtts import gTTS
import os

def text_to_audio(text, filename):
    language = 'es'
    speech = gTTS(text=text, lang=language, slow=False)
    speech.save(filename)


text_to_audio(final_output, f"{tema}.mp3")